In [ ]:
import sys,os
USER = os.environ['USERPROFILE']
sys.path.insert(1,f'{USER}\\PycharmProjects\\my_utility')

from utility_code.my_utility import *
from utility_code.python_lib_essential import *



In [ ]:

print('With the same seed')
np.random.seed(31415) 
print(np.random.random((2,2)))
# np.random.seed(31415) # reset the random seed back to 31415
print(np.random.random((2,2)))
np.random.seed(31415) 
print(np.random.random((2,2)))
# np.random.seed(31415) # reset the random seed back to 31415
print(np.random.random((2,2)))

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
X = np.array([[1, 2], [3, 4], [3, 4], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 0,0,1, 1])
skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(X, y)

print(skf)  

for train_index, test_index in skf.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

In [ ]:
x = True
y = 'a'
if x and y is not None: 
    print("yes")
else:
    print("No")

In [ ]:
if x and y is not None: 
    print("yes")
if not x or y is None: 
    print("No")

In [ ]:
import torch

In [ ]:
x = torch.rand((3,2))
x

In [ ]:
x.size(1)

In [ ]:
x = [i for i in range(1, 3000)]
y = [i for i in range(2000, 5000)]
%timeit [len(set(x).intersection(set(y)))]

In [ ]:
import numpy as np 
x = np.arange(3000)
y = np.arange(3000) + 2000
%timeit np.intersect1d(x, y)

In [ ]:
%timeit np.in1d(x,y, assume_unique=True)

In [ ]:
%timeit x.dot(y)

In [35]:
import networkx as nx
G = nx.complete_graph(5)
# preds = nx.jaccard_coefficient(G, [(0, 1), (2, 3)])
# G = nx.Graph()
preds = nx.jaccard_coefficient(G, [(2, 3), (0, 1)])
for u, v, p in preds:
#     print('(%d, %d) -> %.8f' % (u, v, p))
    G.add_weighted_edges_from([(u,v,100)], weight='jaccard_coeff')

print(G.edges)
# '(0, 1) -> 0.60000000'
# '(2, 3) -> 0.60000000'

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [37]:
nx.to_numpy_matrix(G, weight='jaccard_coeff')

matrix([[  0., 100.,   1.,   1.,   1.],
        [100.,   0.,   1.,   1.,   1.],
        [  1.,   1.,   0., 100.,   1.],
        [  1.,   1., 100.,   0.,   1.],
        [  1.,   1.,   1.,   1.,   0.]])

In [23]:
G.add_weighted_edges_from([(0,1,100)], weight='jaccard_coeff')

In [27]:
G.get_edge_data(0,1)

{'weight': 100, 'jaccard_coeff': 100}